In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f


spark = SparkSession.builder.appName('SparkByExamples.com')\
    .config('spark.driver.bindAddress','localhost')\
    .config("spark.ui.port","4051")\
    .config("spark.driver.memory","5g")\
    .getOrCreate()


23/02/13 10:40:45 WARN Utils: Your hostname, Chaturvedi_PC resolves to a loopback address: 127.0.1.1; using 172.27.71.32 instead (on interface eth0)
23/02/13 10:40:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/02/13 10:40:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
train_input_path = '/home/abhay/work/dream11/processed_output/training_rows'

In [3]:
train_df = spark.read.parquet(train_input_path)

23/02/13 10:40:54 WARN SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.


In [4]:
limit_df = train_df.limit(100)

In [5]:
test_df = limit_df.toPandas()

23/02/13 10:40:58 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
test_df[['batter_run_sum', 'dismissals', 'balls_faced',
            'boundary_count', 'six_count',
            'total_run_sum', 'wicket_sum', 'deliveries', 'maiden_count',
            'fielding_wicket_sum','fantasy_points']]

,batter_run_sum,dismissals,balls_faced,boundary_count,six_count,total_run_sum,wicket_sum,deliveries,maiden_count,fielding_wicket_sum,fantasy_points
0,0,0,0,0,0,29,2,26,0,0,54
1,0,0,0,0,0,0,0,0,0,1,11
2,2,1,3,0,0,0,0,0,0,0,4
3,0,1,2,0,0,34,0,18,0,0,-4
4,0,0,0,0,0,14,1,13,0,0,29
...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,1,0,0,29,0,27,0,1,5
96,6,0,13,0,0,21,2,28,0,0,58
97,0,0,0,0,0,0,0,0,0,1,11
98,2,1,14,0,0,0,0,0,0,0,0
